In [9]:
import keras as K
from tensorflow.keras.models import load_model
import pywt
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import shutil
# os.environ["CUDA_VISIBLE_DEVICES"]=""
# print(os.listdir("/media/liem/hai/haihh/dataset/classify/classify/train"))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)  #chay server thi comment
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default

def scale(x, mode=0, axis=None):
    xmax, xmin = x.max(), x.min()
    x = (x - xmin)/(xmax - xmin)
    if mode != 0:
        x = (x - 0.5) * 2
    return x

def DWT(img):
    coeffs2 = pywt.dwt2(img, 'bior1.3')
    LL, (LH, HL, HH) = coeffs2     #500x402x4
    norm_LL = scale(LL, 0, 2)
    norm_LH = scale(LH, -1, 2)
    norm_HL = scale(HL, -1, 2)
    return norm_LL, norm_LH, norm_HL

model = load_model("weights-25-0.9087.hdf5")
PATH = "test_recapture"

GT = 1
batch_size = 16
list_file = os.listdir(PATH)
# print(list_file)
step = len(list_file) // batch_size
print(step)
count = 0
for iteration in range(step + 1):
    batch_ll = []
    batch_lh = []
    batch_hl = []
    files = []
    for item in range(batch_size):
        idx = item + iteration * batch_size
        if idx >= len(list_file):
            continue
        img_path = os.path.join(PATH, list_file[idx])
        img = cv2.imread(img_path)
        ll0, lh0, hl0 = DWT(img)
        batch_ll.append(ll0)
        batch_lh.append(lh0)
        batch_hl.append(hl0)
        
#         img = cv2.resize(img, (224, 224))
#         batch.append(img)
        files.append(list_file[idx])
    if(len(batch_ll) == 0):
        continue
    batch_ll = np.asarray(batch_ll)
    batch_lh = np.asarray(batch_lh)
    batch_hl = np.asarray(batch_hl)
    
    res = model.predict([batch_ll, batch_lh, batch_hl])
    print(res)
    class_res = np.argmax(res, axis=1)
    for iii in range(len(class_res)):
        if(class_res[iii] != GT):
            print("WRONG: {} ---- {}".format(files[iii], class_res[iii]))
            shutil.copy(os.path.join(PATH, files[iii]), os.path.join("wrong", files[iii]))
            count += 1
print("COUNT: ", count)
print("RESULT: {}".format(count/len(list_file)))
cv2.destroyAllWindows()
# print(res)

1
[[1.0000000e+00 6.9945578e-09]
 [1.0000000e+00 7.1988768e-18]
 [1.0000000e+00 1.1208689e-17]
 [1.0000000e+00 3.4045591e-10]
 [1.0000000e+00 4.3896901e-13]
 [1.0000000e+00 7.7841864e-21]
 [9.8104972e-01 1.8950211e-02]
 [1.0000000e+00 1.9985809e-09]
 [9.8016775e-01 1.9832209e-02]
 [1.0000000e+00 9.9073411e-26]
 [3.1085649e-01 6.8914354e-01]
 [1.0000000e+00 2.3639506e-19]
 [9.8899007e-01 1.1009884e-02]
 [1.0000000e+00 5.6962665e-15]
 [1.0000000e+00 6.0987926e-10]
 [1.0000000e+00 5.1555362e-24]]
WRONG: MVIMG_20190725_121952_786.jpg ---- 0
WRONG: MVIMG_20190725_121915_000.jpg ---- 0
WRONG: MVIMG_20190725_121051_938.jpg ---- 0
WRONG: MVIMG_20190725_121926_411.jpg ---- 0
WRONG: final_image_Wed Sep 18 15:55:59 GMT+07:00 2019.jpg ---- 0
WRONG: MVIMG_20190725_121944_289.jpg ---- 0
WRONG: MVIMG_20190725_121934_740.jpg ---- 0
WRONG: MVIMG_20190725_121056_215.jpg ---- 0
WRONG: IMG_20190725_130928.jpg ---- 0
WRONG: MVIMG_20190725_121904_173.jpg ---- 0
WRONG: final_image_Wed Sep 18 15:52:50 GMT+07: